<a href="https://colab.research.google.com/github/pponvannan/AIMLMadeEasy/blob/master/grofers_scrapping_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [0]:
def split_unitprice(qty):
    final_qty=''
    final_Unit=''
    combo=''
    qty_unit=''
    qtyarr= qty.split(' ')
    if len(qtyarr)==2:
        final_qty=qtyarr[0]
        qty_unit=qtyarr[1]
        qtyarrm=final_qty.split('x')
        if len(qtyarrm)==2:
            final_Unit=qtyarrm[0]
            final_qty=qtyarrm[1]
            combo='Y'
        else:
            final_Unit=1
            final_qty=qtyarrm[0]
            combo='N'
    else:
        final_qty=qty
        final_Unit=1
        combo='N'
        qty_unit=''
    return final_qty,final_Unit,combo,qty_unit

In [0]:
import re
# grofers 
def get_data_grofers(category,ptype, pageNo):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64;     x64; rv:66.0) Gecko/20100101 Firefox/66.0", \
               "Accept-Encoding":"gzip, deflate",     "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    #print("https://grofers.com/s/?q="+ptype)
    r = requests.get("https://grofers.com/s/?q="+ptype, headers=headers)
    
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup.prettify())
    
    #x=soup.find('div', attrs={'class':'plp-product'})
    #print(x)
    
    p_product_df = pd.DataFrame(columns=['source','product_name','brand_name','mrp','selling_price','category','sub_category','is_combo','quandity','quandity_units','unit','price_per_unit'])
    
    for d in soup.findAll('div', attrs={'class':'plp-product'}):
        name = d.find('div', attrs={'class':'plp-product__name--box'})
        price = d.find('span', attrs={'class':'plp-product__price--new'})
        mrp = d.find('div', attrs={'class':'plp-product__price--old display--inline-block@mobile'})
        qty = d.find('div', attrs={'class':'plp-product__quantity'})
    
        #print(rating.text)   
        if name is not None:
            final_name = name.text
            brand=final_name.split(' ')[0]
        else:
            final_name = "unknown-product"
       
        if price is not None:
            final_price = price.text
            final_price = final_price[1:]
        else:
            final_price = '0'
        
        if mrp is not None:
            final_mrp = mrp.text
            final_mrp = final_mrp[1:]
        else:
            final_mrp = final_price      

        if qty is not None:
          t_qty=qty.text
          final_qty,final_Unit,combo,qty_unit=split_unitprice(t_qty)
        unit_price = float(final_price)/int(final_Unit)
        p_product_df = p_product_df.append({'source':'Grofers','product_name': final_name,'brand_name':brand,'mrp':final_mrp,'selling_price':final_price,'category':category,'sub_category':ptype,'is_combo':combo,'quandity':final_qty,'quandity_units':qty_unit,'unit':final_Unit,'price_per_unit':unit_price}, ignore_index=True)
      #           #print(ptype,final_name, final_price, final_qty)
      #   #product_df.append({'Name': 'acer','Price':'1000$','Rating':'4 out of 5'}, ignore_index=True)
      #  # p_product_df = p_product_df.append({'source':'Grofers','Category':ptype,'ProductName': final_name,'MRP':final_mrp,'ActualPrice':final_price,'Qty':final_qty,'QtyUnit':qty_unit,'Unit':final_Unit,'Combo':combo,'UnitPrice':unit_price}, ignore_index=True)
      #    p_product_df = p_product_df.append({'source':'Grofers','product_name': final_name,'brand_name':'brand name','mrp':final_mrp,'selling_price':final_price,'category':'category','sub_category':ptype,'is_combo':combo,'quandity':final_qty,'quandity_units':qty_unit,'unit':final_Unit,'price_per_unit':unit_price}, ignore_index=True)
        
    return p_product_df

In [47]:
data_df = pd.DataFrame(columns=['source','product_name','brand_name','mrp','selling_price','category','sub_category','is_combo','quandity','quandity_units','unit','price_per_unit'])
no_pages = 1

category_dict ={'grocery_staples':['Pulses', 'Atta & Other Flours','Rice & Other Grains','Dry Fruits & Nuts','Edible Oils','Ghee & Vanaspati','Spices','Salt & Sugar','Organic Staples']}
category_dict.update({'personal_care':['Safety Must Haves', 'Bath & Body','Hair Care','Skin Care','Oral Care','Fragrances','Face Care','Feminine Hygiene','Health And Wellness','Cosmetics']})
category_dict.update({'household_items': ['Laundry Detergents', 'Dishwashers','Cleaners','Repellents','Cleaning Tools & Brushes','Pooja Needs','Home & Car Fresheners','Tissues & Disposables','Disinfectants']})
category_dict.update({'home_kichen':['Cookware', 'Storage & Containers','Kitchen Tools & Accessories','Dining & Serving','Cleaning Equipment','Electrical Goods & Accessories','Stationery & Magazines','Gas Stoves']})
category_dict.update({'biscuts_snacks':['Chocolates & Candies', 'Biscuits & Cookies','Namkeen & Snacks','Chips & Crisps','Sweets']})
category_dict.update({'Beverages':['Cold Drinks', 'Juices & Drinks','Tea & Coffee','Health & Energy Drinks','Water & Soda','Milk Drinks']})
category_dict.update({'breakfast_dairy':['Milk & Milk Products', 'Bread & Eggs','Paneer & Curd','Butter & Cheese','Breakfast Cereal & Mixes','Jams, Honey & Spreads']})
category_dict.update({'baby_care':['Baby Food','Diapers and Wipes','Baby Skin','Hair Care','Baby Accessories']})
category_dict.update({'instant_food':['Noodles', 'Sauces','Vermicelli','Ketchups','Jams', 'Honey','Spreads','Pasta','Soups','Ready Made Meals','Pickles','Chutneys','Baking & Dessert Items']})

for x,y in category_dict.items():
  ptype_list=y
  print(y)
  for z in ptype_list:
    for i in range(no_pages):
        data_df = data_df.append(get_data_grofers(x,z,i),ignore_index=True)

['Pulses', 'Atta & Other Flours', 'Rice & Other Grains', 'Dry Fruits & Nuts', 'Edible Oils', 'Ghee & Vanaspati', 'Spices', 'Salt & Sugar', 'Organic Staples']
['Safety Must Haves', 'Bath & Body', 'Hair Care', 'Skin Care', 'Oral Care', 'Fragrances', 'Face Care', 'Feminine Hygiene', 'Health And Wellness', 'Cosmetics']
['Laundry Detergents', 'Dishwashers', 'Cleaners', 'Repellents', 'Cleaning Tools & Brushes', 'Pooja Needs', 'Home & Car Fresheners', 'Tissues & Disposables', 'Disinfectants']
['Cookware', 'Storage & Containers', 'Kitchen Tools & Accessories', 'Dining & Serving', 'Cleaning Equipment', 'Electrical Goods & Accessories', 'Stationery & Magazines', 'Gas Stoves']
['Chocolates & Candies', 'Biscuits & Cookies', 'Namkeen & Snacks', 'Chips & Crisps', 'Sweets']
['Cold Drinks', 'Juices & Drinks', 'Tea & Coffee', 'Health & Energy Drinks', 'Water & Soda', 'Milk Drinks']
['Milk & Milk Products', 'Bread & Eggs', 'Paneer & Curd', 'Butter & Cheese', 'Breakfast Cereal & Mixes', 'Jams, Honey & Sp

In [0]:
#data_df.to_csv('grofer_products.csv')
data_df = pd.read_csv("grofer_products.csv")

In [83]:
print(data_df.shape)
#data_df = data_df[~data_df['Qty'].isin(['+'])]
data_df.drop(data_df[data_df['quandity'].str.split().str.len() > 2].index, inplace = True) 
print(data_df.shape)


(2039, 13)
(1984, 13)


In [0]:
data_df["productname_qty"] = data_df["product_name"] + data_df["quandity"]


In [0]:
data_df["quandity"] = pd.to_numeric(data_df["quandity"],errors='coerce')

In [86]:
print(data_df.shape)

(1984, 14)


In [87]:
# Validating Units
print(data_df['quandity_units'].unique())
#Validating the Category
print(data_df['category'].unique())


['g' 'kg' nan 'gm' 'l' 'unit' 'm' 'ml' 'units' 'lt']
['grocery_staples' 'personal_care' 'household_items' 'home_kichen'
 'biscuts_snacks' 'Beverages' 'breakfast_dairy' 'baby_care' 'instant_food']


In [0]:
def missing_values_tables(df):
  mis_val = df.isnull().sum()
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
  mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n There are " + str(mis_val_table_ren_columns.shape[0]) +  " columns that have missing values.")
  return mis_val_table_ren_columns


In [89]:
missing_values_tables(data_df)

Your selected dataframe has 14 columns.
 There are 2 columns that have missing values.


,Missing Values,% of Total Values
quandity_units,37,1.9
quandity,34,1.7


In [90]:
print(len(data_df))
data_df = data_df.dropna(how='any')
print(len(data_df))

1984
1946


In [91]:
# sorting by first name 
data_df.sort_values("productname_qty", inplace = True) 
print('Before Dropping duplicate values')
print(len(data_df))
data_df.drop_duplicates(subset=["productname_qty"],keep = False, inplace = True)
print('After Dropping duplicate values')
print(len(data_df))

Before Dropping duplicate values
1946
After Dropping duplicate values
923


There is no duplicate values by Product Name and Category.

In [0]:
data_df.drop("productname_qty", axis=1, inplace=True)

In [0]:
data_df.to_csv('grofer_products_New.csv')